在TinyStories上训练以字节为单位的BPE分词器。

##### 预分词的并行化
预分词步骤将成为一个重要的性能瓶颈，故可以使用built-in library `multiprocessing`并行化来加速。

在并行化预分词的实现中，将语料分块时要注意块边界出现在special token的开始处。提供的预分词示例代码可以用于找到块的边界，找到后即可用于并行时的任务分配。这种分块策略永远是有用的，因为我们永远不会希望跨文档的合并操作。本作业中无需担心语料中没有`<|endoftext|>`导致块过大。

##### 去掉special token
在用`re.finditer`正则预分词之前，去掉special token（无论你处理整个语料还是某个块）。

使用`re.split`和` "|" ⌋.join(special_tokens)`。(with careful use of re.escape since | may occur in the special tokens)

这部分对应`test_train_bpe_special_tokens`。

##### 优化合并步骤
最朴素的合并算法太慢了，因为每次合并都要去看所有当前的字节对（或token对）。However, the only pair counts that change after each merge are those that overlap with the merged pair. Thus, BPE training speed can be improved by indexing the counts of all pairs and incrementally updating these counts, 而非一直遍历并计数所有对。这样能快很多，尽管这里不能并行。

##### 对于低算力：Profiling
用`cProfile`或`scalene`等工具分析性能瓶颈并优化它们。

##### 对于低算力：Downscaling/降尺度
先在数据集的一小部分上实验，例如在验证集上训练，大小约百分之一。选取小的子集时也要注意不能太小。

### Problem (train_bpe): BPE Tokenizer Training (15 points)
交付内容：写一个函数，输入文本文件路径，训练字节为单位的bpe分词器。

输入参数：
- `input_path`：`str`字符串，数据文本文件路径
- `vocab_size`：`int`正整数，最终词表大小，包含最初的各种字节、合并出的内容、special tokens
- `special_tokens`：`list[str]`字符串列表，不影响bpe训练

返回参数：
- `vocab`：`dict[int,bytes]`字节串的词典映射，每个字节串（即最终得到的token）编一个整数序号（即token ID）。
- `merges`：`list[tuple[bytes, bytes]]`字节串二元组的列表，训练过程中合并token的记录，按合并顺序从前往后列出。

最终目标：将写好的内容填入`adapters.py`文件中`run_train_bpe`函数处，运行`uv run pytest tests/test_train_bpe.py`进行测试。

附加可选目标：把关键部分用cpp（用cppyy）或rust（用PyO3）来写。如果你要这么做，请注意区分哪些操作需要复制内存，哪些是直接从 Python 内存中读取。另外，请务必留下构建说明，或者确保项目仅使用 `pyproject.toml` 文件就能完成构建。

另外注意给定的GPT-2的正则模板未必所有引擎中都支持，即使支持可能也很慢。已经验证`Oniguruma`库的速度相当快，并且支持负向先行断言（negative lookahead），但Python的`regex`也并不逊色。

In [46]:
from io import BytesIO
from pprint import pprint
from cs336_basics.pretokenization_example import *
import regex as re

In [47]:
# def path_to_bytesfile(p:str, n:int = -1) -> BytesIO:
#     if n == -1:
#         with open(p, 'rb') as f:
#             text = f.read()
#         return BytesIO(text)
#     else:
#         with open(p, 'rb') as f:
#             text = f.read(n)
#         return BytesIO(text)
    
# test = path_to_bytesfile("../data/TinyStoriesV2-GPT4-valid.txt")
# print(test)
# type(test)

In [48]:
def path_to_chunks_bytes(p:str, n_parallel: int) -> list[bytes] :
    res = []
    with open(p,"rb") as f:
        num_processes = n_parallel
        boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")
        for start, end in zip(boundaries[:-1], boundaries[1:]):
            f.seek(start)
            chunk = f.read(end - start) #.decode("utf-8", errors="ignore")
            res.append(chunk)
    return res

testchunks = path_to_chunks_bytes(p="../data/testinput.txt",n_parallel=4)
for i, chunk in enumerate(testchunks):
    print(f"第{i}段的前100个字节：")
    print(chunk[:100])


第0段的前100个字节：
b'the cat on the mat\nWhat a good cat!\nzhe shi yi ge ce shi yong li\n<|endoftext|>\nmad mad mad mad mad m'
第1段的前100个字节：
b"<|endoftext|>adjfjf\n4564129d|w1dsvdsv|davniuab asdkvhiaudva<|endoftext|>ef,l;/a f.mbsop'fdbifadfdafd"
第2段的前100个字节：
b'<|endoftext|>\nmad mad mad mad mad mad\nmadam madam madam madam madam madam\ndamn damn damn damn damn d'
第3段的前100个字节：
b"<|endoftext|>adjfjf\n4564129d|w1dsvdsv|davniuab asdkvhiaudva<|endoftext|>ef,l;/a f.mbsop'fdbifadfdafd"


In [49]:
# 对于每段文本，去掉特殊token并切分的过程
def pre_tokenization_for_chunk(text_chunk_bytes:bytes, special_tokens: list[str]) -> list[str]:
    text_chunk = text_chunk_bytes.decode("utf-8")
    escaped_special_tokens = [re.escape(t) for t in special_tokens]
    escaped_special_tokens_in_one_str = "|".join(escaped_special_tokens)
    print(escaped_special_tokens_in_one_str)
    splited_text = re.split(escaped_special_tokens_in_one_str,text_chunk)
    pre_tokenization = []
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    for doc in splited_text:
        it = re.finditer(PAT,doc)
        for match in it:
            pre_tokenization.append(match.group())
    return pre_tokenization

testpretok = []
for i, chunk in enumerate(testchunks) : 
    print(f"------第{i}段测试文本信息：------")
    print("前100字符：",chunk[:100])
    res = pre_tokenization_for_chunk(chunk,["<|endoftext|>"])
    print("预分词结果前10个：")
    print(res[:10])
    testpretok.append(res)

print("总体预分词结果：",testpretok)

------第0段测试文本信息：------
前100字符： b'the cat on the mat\nWhat a good cat!\nzhe shi yi ge ce shi yong li\n<|endoftext|>\nmad mad mad mad mad m'
<\|endoftext\|>
预分词结果前10个：
['the', ' cat', ' on', ' the', ' mat', '\n', 'What', ' a', ' good', ' cat']
------第1段测试文本信息：------
前100字符： b"<|endoftext|>adjfjf\n4564129d|w1dsvdsv|davniuab asdkvhiaudva<|endoftext|>ef,l;/a f.mbsop'fdbifadfdafd"
<\|endoftext\|>
预分词结果前10个：
['adjfjf', '\n', '4564129', 'd', '|', 'w', '1', 'dsvdsv', '|', 'davniuab']
------第2段测试文本信息：------
前100字符： b'<|endoftext|>\nmad mad mad mad mad mad\nmadam madam madam madam madam madam\ndamn damn damn damn damn d'
<\|endoftext\|>
预分词结果前10个：
['\n', 'mad', ' mad', ' mad', ' mad', ' mad', ' mad', '\n', 'madam', ' madam']
------第3段测试文本信息：------
前100字符： b"<|endoftext|>adjfjf\n4564129d|w1dsvdsv|davniuab asdkvhiaudva<|endoftext|>ef,l;/a f.mbsop'fdbifadfdafd"
<\|endoftext\|>
预分词结果前10个：
['adjfjf', '\n', '4564129', 'd', '|', 'w', '1', 'dsvdsv', '|', 'davniuab']
总体预分词结果： [['the', ' cat', ' on', ' the

In [50]:
# 将多组预分词结果合并成一个列表，并转为bytes类型
def get_all_pretoken_bytes(pre_tokens:list[list[str]])->list[bytes]:
    res = [tok.encode("utf-8") for trunks in pre_tokens for tok in trunks]
    return res

pre_tokens_bytes = get_all_pretoken_bytes(testpretok)
print("整合所有预分词结果并转为bytes，前20个：\n",pre_tokens_bytes[:20])

整合所有预分词结果并转为bytes，前20个：
 [b'the', b' cat', b' on', b' the', b' mat', b'\n', b'What', b' a', b' good', b' cat', b'!', b'\n', b'zhe', b' shi', b' yi', b' ge', b' ce', b' shi', b' yong', b' li']


In [51]:
# # 将一串bytes转为整数组
# def bytes_to_ints(input:bytes)->list[int]:
#     res = list(input)
#     return res

# print("再写一遍测试文本，前20字节\n",chunk_bytes[:20])
# print("直接将bytes按字节翻译成整数\n",bytes_to_ints(chunk_bytes[:20]))

In [52]:
pre_token_ints = [list(pre_token) for pre_token in pre_tokens_bytes]
print("测试文本转为整数组，前十个pretoken\n",pre_token_ints[:10])

测试文本转为整数组，前十个pretoken
 [[116, 104, 101], [32, 99, 97, 116], [32, 111, 110], [32, 116, 104, 101], [32, 109, 97, 116], [10], [87, 104, 97, 116], [32, 97], [32, 103, 111, 111, 100], [32, 99, 97, 116]]


In [53]:
from collections import defaultdict
type IntPairPositionDict = dict[tuple[int,int],list[tuple[int,int]]]
type IntBytesDict = dict[int,bytes]

def create_token_pair_dict_int(pre_token_ints:list[list[int]]) -> IntPairPositionDict:
    res = defaultdict(list)
    for ptid, pt in enumerate(pre_token_ints):
        for n in range(len(pt)-1):
            res[(pt[n],pt[n+1])].append((ptid,n))
    # return res
    return dict(res) # 回到普通dict

testdict = create_token_pair_dict_int(pre_token_ints)
print(testdict)

{(116, 104): [(0, 0), (3, 1), (74, 0), (77, 1)], (104, 101): [(0, 1), (3, 2), (12, 1), (74, 1), (77, 2), (86, 1)], (32, 99): [(1, 0), (9, 0), (16, 0), (75, 0), (83, 0), (90, 0)], (99, 97): [(1, 1), (9, 1), (75, 1), (83, 1)], (97, 116): [(1, 2), (4, 2), (6, 2), (9, 2), (75, 2), (78, 2), (80, 2), (83, 2)], (32, 111): [(2, 0), (76, 0)], (111, 110): [(2, 1), (18, 2), (76, 1), (92, 2)], (32, 116): [(3, 0), (77, 0)], (32, 109): [(4, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (78, 0), (97, 0), (98, 0), (99, 0), (100, 0), (101, 0), (104, 0), (105, 0), (106, 0), (107, 0), (108, 0)], (109, 97): [(4, 1), (22, 0), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (29, 0), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (78, 1), (96, 0), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (103, 0), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1)], (87, 104): [(6, 0), (80, 0)], (104, 97): [(6, 1), (80, 1)], (32, 97): [(7, 0), (62, 0), (81, 0), (136, 0)], (32, 1

In [ ]:
class BpeManager:
    data: list[list[int]]
    pos_dict: IntPairPositionDict
    vocab_dict: IntBytesDict
    merge_list:list[tuple[bytes,bytes]]

    def __init__(self, pre_token_ints:list[list[int]], special_tokens:list[str]) -> None:
        self.data = pre_token_ints
        self.pos_dict = create_token_pair_dict_int(pre_token_ints)
        self.vocab_dict = {n: bytes([n]) for n in range(256)}
        for st in special_tokens:
            self.vocab_dict[len(self.vocab_dict)] = st.encode("utf-8")
        self.merge_list = []
    
    def get_max_token_pair_int(self) -> tuple[int,int]:
        # if not self.pos_dict:
        #     print("pos_dict为空，无法继续合并，结束合并过程")
        #     self.end = True
        #     return (-1,-1)
        maxpair = max(
            self.pos_dict, 
            key = lambda k : (
                len(self.pos_dict[k]), 
                self.vocab_dict[k[0]], 
                self.vocab_dict[k[1]]
            )
        )
        print("---寻找出现最多的token对---\n",
            f"出现最多的token对是{maxpair}，即{self.vocab_dict[maxpair[0]]}与{self.vocab_dict[maxpair[1]]}\n",
            f"出现了{len(self.pos_dict[maxpair])}次\n",
            f"出现在这些位置：{self.pos_dict[maxpair]}\n",
            "---寻找结束---\n")
        return maxpair
    
    def clear_pos_dict(self,pos:tuple[int,int]):
        # 清理第m个词的所有pair
        m = pos[0]
        zipped = zip(self.data[m][:-1],self.data[m][1:])
        for pair in zipped :
            assert pair in self.pos_dict, f"在清理时，发现pair {pair} 不在pos_dict中，pos_dict中只有{self.pos_dict.keys()}"
            for pairpos in self.pos_dict[pair]:
                if pairpos[0] == m and pairpos in self.pos_dict[pair]:
                    self.pos_dict[pair].remove(pairpos)
            if self.pos_dict[pair] == []:
                del self.pos_dict[pair]
    
    def rebuild_pos_dict(self,pos:tuple[int,int]):
        # 填充第m个词的所有pair
        m = pos[0]
        zipped = zip(self.data[m][:-1],self.data[m][1:])
        for i, pair in enumerate(zipped) :
            self.pos_dict.setdefault(pair,[]).append((m,i))

    # 一次merge的全过程
    def merge(self, new_pair_id:tuple[int,int]) -> None:
        # 在vocab_dict中加入新token nt = lt + rt
        # 在merge中加入新merge (lt,rt)
        li = new_pair_id[0]
        ri = new_pair_id[1]
        lt = self.vocab_dict[li]
        rt = self.vocab_dict[ri]
        ni = len(self.vocab_dict)
        nt = lt + rt
        self.vocab_dict[ni] = nt
        self.merge_list.append((lt,rt))

        print("---开始合并---\n",
            "本次合并情况：\n",
            f"{li}--{lt}与{ri}--{rt}合并得到{nt}，编号为{ni}")
        
        # 从pos_dict中找到所有 (li,ri) 的位置 (m,n)
        assert (li,ri) in self.pos_dict
        pos_list = self.pos_dict[(li,ri)]
        print("他们出现在这些位置：",f"{self.pos_dict[(li,ri)]}")

        while pos_list != []:
            pos = pos_list.pop()
            m = pos[0]
            n = pos[1]
            self.clear_pos_dict(pos)
            # print("删改前的序列：",self.data[m])
            self.data[m].pop(n+1)
            self.data[m][n] = ni
            # print("删改后的序列：",self.data[m])
            self.rebuild_pos_dict(pos)

        print("---合并结束---\n")

    def quick_look(self):
        pass
        print("------状态速览------")
        print("当前的data：")
        print(self.data)
        # print([bytes(pt) for pt in self.data])
        print([[self.vocab_dict[i] for i in j] for j in self.data])
        print("当前的token对位置索引pos_dict：")
        print(self.pos_dict)
        print([[(self.vocab_dict[k[0]],self.vocab_dict[k[1]]),v] for k,v in self.pos_dict.items()])
        print("当前的token表vocab_dict（最后5项）：")
        print(list(self.vocab_dict.items())[-5:])
        print("当前的合并列表：")
        print(self.merge_list)
        print("------状态速览------")
        print()


In [ ]:
def main_bpe(
    pre_token_ints: list[list[int]],
    vocab_size: int,
    special_tokens: list[str]
    )->tuple[
        dict[int,bytes],                # vocab
        list[tuple[bytes,bytes]]        # merge
    ]:

    manager = BpeManager(pre_token_ints=pre_token_ints, special_tokens=special_tokens)

    while len(manager.vocab_dict) < vocab_size:
        print("当前词表大小：",len(manager.vocab_dict))
        manager.quick_look()
        if manager.pos_dict == {}:
            print("不再有任何相邻token对，合并过程提前结束")
            break
        maxpair = manager.get_max_token_pair_int()
        print("获得的最大token对：",
              manager.vocab_dict[maxpair[0]],
              manager.vocab_dict[maxpair[1]]
              )
        manager.merge(maxpair)

    return (manager.vocab_dict, manager.merge_list)


(vocab,merge) = main_bpe(pre_token_ints=pre_token_ints, vocab_size=5000, special_tokens=["<|endoftext|>"])

当前词表大小： 257
------状态速览------
当前的data：
[[116, 104, 101], [32, 99, 97, 116], [32, 111, 110], [32, 116, 104, 101], [32, 109, 97, 116], [10], [87, 104, 97, 116], [32, 97], [32, 103, 111, 111, 100], [32, 99, 97, 116], [33], [10], [122, 104, 101], [32, 115, 104, 105], [32, 121, 105], [32, 103, 101], [32, 99, 101], [32, 115, 104, 105], [32, 121, 111, 110, 103], [32, 108, 105], [10], [10], [109, 97, 100], [32, 109, 97, 100], [32, 109, 97, 100], [32, 109, 97, 100], [32, 109, 97, 100], [32, 109, 97, 100], [10], [109, 97, 100, 97, 109], [32, 109, 97, 100, 97, 109], [32, 109, 97, 100, 97, 109], [32, 109, 97, 100, 97, 109], [32, 109, 97, 100, 97, 109], [32, 109, 97, 100, 97, 109], [10], [100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [32, 100, 97, 109, 110], [10], [97, 115, 100], [124], [102, 97, 100, 102, 97, 115, 100, 102], [124], [97], [32, 100, 102,

AssertionError: 在清理时，发现pair (100, 102) 不在pos_dict中，pos_dict中只有dict_keys([(116, 104), (104, 101), (32, 99), (99, 97), (97, 116), (32, 111), (111, 110), (32, 116), (87, 104), (104, 97), (32, 97), (32, 103), (103, 111), (111, 111), (111, 100), (122, 104), (32, 115), (115, 104), (104, 105), (32, 121), (121, 105), (103, 101), (99, 101), (121, 111), (110, 103), (32, 108), (108, 105), (97, 100), (97, 115), (115, 100), (102, 97), (100, 106), (106, 102), (102, 106), (52, 53), (53, 54), (54, 52), (52, 49), (49, 50), (50, 57), (100, 115), (115, 118), (118, 100), (118, 110), (110, 105), (105, 117), (117, 97), (97, 98), (100, 107), (107, 118), (118, 104), (105, 97), (97, 117), (117, 100), (100, 118), (118, 97), (101, 102), (59, 47), (32, 102), (109, 98), (98, 115), (115, 111), (111, 112), (102, 100), (100, 98), (98, 105), (105, 102), (257, 102), (257, 118), (259, 258), (259, 100), (260, 116), (260, 100), (260, 258), (97, 263), (263, 257), (102, 263), (263, 97), (32, 263), (115, 263)])

In [ ]:
print(list(vocab.items())[-10:])
print(merge)

In [ ]:
def my_bpe(input_path:str, vocab_size:int, special_tokens:list[str]):
    # 从路径读取文件，转为文件指针。注意读出来的是bytes类型
    limited_f = path_to_bytesfile(input_path, 4096)

    # 定义分块参数并进行分块，获得边界
    num_processes = 4
    split_special_token = b"<|endoftext|>"
    boundaries = find_chunk_boundaries(limited_f, num_processes, split_special_token)
    print(boundaries)

    
    # limited_f.seek(4075)
    # data = limited_f.read(100)
    # print(data)

    # 对每一块分别处理
    for bound1, bound2 in zip(boundaries[:-1], boundaries[1:]):
        #print(bound1, bound2)
        limited_f.seek(bound1)
        chunk = limited_f.read(bound2 - bound1) #每个chunk是bytes类型
        print(len(chunk))
    
    # return vocab, merges

my_bpe("../data/TinyStoriesV2-GPT4-valid.txt", 100, ["<|endoftext|>"])

课程提供的pretokenization用法
```python
with open(..., "rb") as f:
    num_processes = 4
    boundaries = find_chunk_boundaries(f, num_processes, b"<|endoftext|>")

    # The following is a serial implementation, but you can parallelize this
    # by sending each start/end pair to a set of processes.
    for start, end in zip(boundaries[:-1], boundaries[1:]):
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")
        # Run pre-tokenization on your chunk and store the counts for each pre-token
```

### Problem (train_bpe_tinystories): BPE Training on TinyStories (2 points)
**(a) 在 TinyStories 数据集上训练字节级 BPE 分词器。词表大小为10000。TinyStories的special token是<|endoftext|>。将训练生成的词表和合并序列化（serialize，即存成一个json之类的文件）到本地。训练过程花费了多少小时，占用了多少内存？词表中最长的词元（token）是什么？这个结果是否合理？**

资源限制：不使用GPU情况下不超过30分钟，不超过30GB内存。

提示：注意`<|endoftext|>`分割了各个文档。在进行bpe合并前要先处理它们。知道以上事实并使用并行预分词可以将时间压缩到两分钟内。

交付内容：一到两句话。

**(b)分析代码性能，哪一步耗时最多？** 交付内容：一到两句话。

下面再在 `OpenWebText` 上训练试试。

### Problem (train_bpe_expts_owt): BPE Training on OpenWebText (2 points)
**(a) 同上题，数据集改为OpenWebText，词表大小改为32000。**

资源限制：不使用GPU情况下不超过12小时，不超过100GB内存。

交付内容：一到两句话。

**(b) 比较两个数据集上训练的分词器的区别。** 交付内容：一到两句话。